In [ ]:
# ============================================================================
# STL-10 UNSUPERVISED LEARNING - COMPLETE PIPELINE
# No Deep Learning - Pure Traditional ML
# ============================================================================

# Install required packages
!pip install numpy matplotlib seaborn scikit-learn scikit-image pandas umap-learn torch torchvision

# Import all libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog, local_binary_pattern
from skimage.color import rgb2gray
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

# ============================================================================
# STEP 1: LOAD STL-10 DATASET
# ============================================================================

from torchvision.datasets import STL10

print("\n" + "="*80)
print("LOADING STL-10 DATASET")
print("="*80)

# Load unlabeled dataset
dataset = STL10(root='./data', split='unlabeled', download=True)

# Sample for computational efficiency
np.random.seed(42)
n_samples = 15000
sample_indices = np.random.choice(len(dataset), n_samples, replace=False)

print(f"Sampling {n_samples} images from {len(dataset)} total unlabeled images...")

# Convert to numpy arrays
images = []
for idx in sample_indices:
    img, _ = dataset[idx]
    images.append(np.array(img))
    if len(images) % 1000 == 0:
        print(f"Loaded {len(images)}/{n_samples} images...")

images = np.array(images)
print(f"\n✅ Dataset loaded successfully!")
print(f"Dataset shape: {images.shape}")
print(f"Image dimensions: {images[0].shape}")

# ============================================================================
# STEP 2: EXPLORATORY DATA ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("EXPLORATORY DATA ANALYSIS")
print("="*80)

# Display sample images
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.ravel()
random_indices = np.random.choice(len(images), 20, replace=False)

for i, idx in enumerate(random_indices):
    axes[i].imshow(images[idx])
    axes[i].axis('off')
    axes[i].set_title(f'Image {idx}', fontsize=10)

plt.suptitle('Sample Images from STL-10 Unlabeled Dataset', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('01_sample_images.png', dpi=300, bbox_inches='tight')
plt.show()

# Analyze color distribution
mean_r = images[:, :, :, 0].mean(axis=(1, 2))
mean_g = images[:, :, :, 1].mean(axis=(1, 2))
mean_b = images[:, :, :, 2].mean(axis=(1, 2))

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(mean_r, bins=50, color='red', alpha=0.7, edgecolor='black')
axes[0].set_title('Red Channel Distribution', fontweight='bold')
axes[0].set_xlabel('Mean Red Value')
axes[0].set_ylabel('Frequency')

axes[1].hist(mean_g, bins=50, color='green', alpha=0.7, edgecolor='black')
axes[1].set_title('Green Channel Distribution', fontweight='bold')
axes[1].set_xlabel('Mean Green Value')
axes[1].set_ylabel('Frequency')

axes[2].hist(mean_b, bins=50, color='blue', alpha=0.7, edgecolor='black')
axes[2].set_title('Blue Channel Distribution', fontweight='bold')
axes[2].set_xlabel('Mean Blue Value')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('02_color_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Mean Red: {mean_r.mean():.2f} ± {mean_r.std():.2f}")
print(f"Mean Green: {mean_g.mean():.2f} ± {mean_g.std():.2f}")
print(f"Mean Blue: {mean_b.mean():.2f} ± {mean_b.std():.2f}")

# ============================================================================
# STEP 3: FEATURE EXTRACTION
# ============================================================================

print("\n" + "="*80)
print("FEATURE EXTRACTION")
print("="*80)

# 3.1 Color Histogram Features
print("\nExtracting color histogram features...")
color_features = []
for i, img in enumerate(images):
    hist_r = np.histogram(img[:,:,0], bins=32, range=(0,256))[0]
    hist_g = np.histogram(img[:,:,1], bins=32, range=(0,256))[0]
    hist_b = np.histogram(img[:,:,2], bins=32, range=(0,256))[0]
    hist_combined = np.concatenate([hist_r, hist_g, hist_b])
    hist_normalized = hist_combined / (hist_combined.sum() + 1e-6)
    color_features.append(hist_normalized)
    if (i+1) % 1000 == 0:
        print(f"  Processed {i+1}/{len(images)} images...")

color_features = np.array(color_features)
print(f"✅ Color histogram features: {color_features.shape}")

# 3.2 HOG Features
print("\nExtracting HOG features...")
hog_features = []
for i, img in enumerate(images):
    gray = rgb2gray(img)
    hog_feat = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), visualize=False, feature_vector=True)
    hog_features.append(hog_feat)
    if (i+1) % 1000 == 0:
        print(f"  Processed {i+1}/{len(images)} images...")

hog_features = np.array(hog_features)
print(f"✅ HOG features: {hog_features.shape}")

# 3.3 Statistical Features
print("\nExtracting statistical features...")
stat_features = []
for i, img in enumerate(images):
    feats = []
    for channel in range(3):
        ch = img[:, :, channel]
        feats.extend([ch.mean(), ch.std(), ch.min(), ch.max(), 
                     np.median(ch), np.percentile(ch, 25), np.percentile(ch, 75)])
    feats.append(img.mean())
    feats.append(img.std())
    stat_features.append(feats)
    if (i+1) % 1000 == 0:
        print(f"  Processed {i+1}/{len(images)} images...")

stat_features = np.array(stat_features)
print(f"✅ Statistical features: {stat_features.shape}")

# 3.4 Texture Features (LBP)
print("\nExtracting texture features (LBP)...")
texture_features = []
for i, img in enumerate(images):
    gray = rgb2gray(img)
    gray = (gray * 255).astype(np.uint8)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins))
    hist = hist / (hist.sum() + 1e-6)
    texture_features.append(hist)
    if (i+1) % 1000 == 0:
        print(f"  Processed {i+1}/{len(images)} images...")

max_len = max(len(f) for f in texture_features)
texture_features = np.array([np.pad(f, (0, max_len - len(f))) for f in texture_features])
print(f"✅ Texture features: {texture_features.shape}")

# 3.5 Combine all features
print("\nCombining all features...")
all_features = np.hstack([color_features, hog_features, stat_features, texture_features])
print(f"✅ Combined features shape: {all_features.shape}")
print(f"Total feature dimensions: {all_features.shape[1]}")

# ============================================================================
# STEP 4: DIMENSIONALITY REDUCTION
# ============================================================================

print("\n" + "="*80)
print("DIMENSIONALITY REDUCTION")
print("="*80)

# Standardize features
print("\nStandardizing features...")
scaler = StandardScaler()
features_scaled = scaler.fit_transform(all_features)

# Apply PCA
print("\nApplying PCA...")
pca = PCA(n_components=0.95, random_state=42)
features_pca = pca.fit_transform(features_scaled)

print(f"Original dimensions: {features_scaled.shape[1]}")
print(f"PCA dimensions: {features_pca.shape[1]}")
print(f"Explained variance: {pca.explained_variance_ratio_.sum():.4f}")

# Visualize explained variance
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(np.cumsum(pca.explained_variance_ratio_), linewidth=2)
plt.xlabel('Number of Components', fontsize=12)
plt.ylabel('Cumulative Explained Variance', fontsize=12)
plt.title('PCA Explained Variance', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.95, color='r', linestyle='--', label='95% threshold')
plt.legend()

plt.subplot(1, 2, 2)
plt.bar(range(1, min(21, len(pca.explained_variance_ratio_)+1)), 
        pca.explained_variance_ratio_[:20], alpha=0.7, edgecolor='black')
plt.xlabel('Principal Component', fontsize=12)
plt.ylabel('Explained Variance Ratio', fontsize=12)
plt.title('Top 20 Principal Components', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('03_pca_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# STEP 5: FIND OPTIMAL NUMBER OF CLUSTERS
# ============================================================================

print("\n" + "="*80)
print("FINDING OPTIMAL NUMBER OF CLUSTERS")
print("="*80)

k_range = range(5, 21)
results = {'k': [], 'inertia': [], 'silhouette': [], 'davies_bouldin': [], 'calinski_harabasz': []}

print("\nEvaluating different values of K...")
for k in k_range:
    print(f"Testing K={k}...", end=' ')
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    labels = kmeans.fit_predict(features_pca)
    
    results['k'].append(k)
    results['inertia'].append(kmeans.inertia_)
    results['silhouette'].append(silhouette_score(features_pca, labels))
    results['davies_bouldin'].append(davies_bouldin_score(features_pca, labels))
    results['calinski_harabasz'].append(calinski_harabasz_score(features_pca, labels))
    
    print(f"Silhouette={results['silhouette'][-1]:.3f}")

cluster_metrics = pd.DataFrame(results)

print("\n" + "="*80)
print("CLUSTERING EVALUATION METRICS")
print("="*80)
print(cluster_metrics.to_string(index=False))

# Plot all metrics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].plot(cluster_metrics['k'], cluster_metrics['inertia'], 'o-', linewidth=2, markersize=8, color='#2E86AB')
axes[0, 0].set_xlabel('Number of Clusters (K)', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Inertia', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Elbow Method', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_xticks(cluster_metrics['k'])

axes[0, 1].plot(cluster_metrics['k'], cluster_metrics['silhouette'], 'o-', linewidth=2, markersize=8, color='#A23B72')
axes[0, 1].set_xlabel('Number of Clusters (K)', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Silhouette Score', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Silhouette Score (Higher = Better)', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_xticks(cluster_metrics['k'])
best_sil_idx = cluster_metrics['silhouette'].idxmax()
axes[0, 1].axvline(x=cluster_metrics.loc[best_sil_idx, 'k'], color='red', linestyle='--', alpha=0.5, label='Optimal K')
axes[0, 1].legend()

axes[1, 0].plot(cluster_metrics['k'], cluster_metrics['davies_bouldin'], 'o-', linewidth=2, markersize=8, color='#F18F01')
axes[1, 0].set_xlabel('Number of Clusters (K)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Davies-Bouldin Index', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Davies-Bouldin Index (Lower = Better)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_xticks(cluster_metrics['k'])
best_db_idx = cluster_metrics['davies_bouldin'].idxmin()
axes[1, 0].axvline(x=cluster_metrics.loc[best_db_idx, 'k'], color='red', linestyle='--', alpha=0.5, label='Optimal K')
axes[1, 0].legend()

axes[1, 1].plot(cluster_metrics['k'], cluster_metrics['calinski_harabasz'], 'o-', linewidth=2, markersize=8, color='#6A994E')
axes[1, 1].set_xlabel('Number of Clusters (K)', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Calinski-Harabasz Score', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Calinski-Harabasz Score (Higher = Better)', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xticks(cluster_metrics['k'])
best_ch_idx = cluster_metrics['calinski_harabasz'].idxmax()
axes[1, 1].axvline(x=cluster_metrics.loc[best_ch_idx, 'k'], color='red', linestyle='--', alpha=0.5, label='Optimal K')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('04_optimal_k_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

optimal_k = int(cluster_metrics.loc[cluster_metrics['silhouette'].idxmax(), 'k'])
print(f"\n🎯 OPTIMAL K = {optimal_k} (based on highest Silhouette Score)")

# ============================================================================
# STEP 6: APPLY MULTIPLE CLUSTERING ALGORITHMS
# ============================================================================

print("\n" + "="*80)
print("APPLYING CLUSTERING ALGORITHMS")
print("="*80)

# K-Means
print(f"\n[1] K-Means Clustering (K={optimal_k})...")
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=500)
labels_kmeans = kmeans.fit_predict(features_pca)
silhouette_kmeans = silhouette_score(features_pca, labels_kmeans)
print(f"    Silhouette Score: {silhouette_kmeans:.4f}")
print(f"    Cluster sizes: {np.bincount(labels_kmeans)}")

# Hierarchical
print(f"\n[2] Hierarchical Clustering (K={optimal_k})...")
hierarchical = AgglomerativeClustering(n_clusters=optimal_k, linkage='ward')
labels_hierarchical = hierarchical.fit_predict(features_pca)
silhouette_hierarchical = silhouette_score(features_pca, labels_hierarchical)
print(f"    Silhouette Score: {silhouette_hierarchical:.4f}")
print(f"    Cluster sizes: {np.bincount(labels_hierarchical)}")

# Gaussian Mixture
print(f"\n[3] Gaussian Mixture Model (K={optimal_k})...")
gmm = GaussianMixture(n_components=optimal_k, random_state=42, n_init=10)
labels_gmm = gmm.fit_predict(features_pca)
silhouette_gmm = silhouette_score(features_pca, labels_gmm)
print(f"    Silhouette Score: {silhouette_gmm:.4f}")
print(f"    Cluster sizes: {np.bincount(labels_gmm)}")

# DBSCAN
print(f"\n[4] DBSCAN Clustering...")
dbscan = DBSCAN(eps=3.5, min_samples=10)
labels_dbscan = dbscan.fit_predict(features_pca)
n_clusters_dbscan = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_noise = list(labels_dbscan).count(-1)
print(f"    Clusters found: {n_clusters_dbscan}")
print(f"    Noise points: {n_noise}")
if n_clusters_dbscan > 1 and n_noise < len(labels_dbscan):
    mask_valid = labels_dbscan != -1
    silhouette_dbscan = silhouette_score(features_pca[mask_valid], labels_dbscan[mask_valid])
    print(f"    Silhouette Score: {silhouette_dbscan:.4f}")
else:
    silhouette_dbscan = 0

# Compare algorithms
print("\n" + "="*80)
print("ALGORITHM COMPARISON")
print("="*80)
comparison = pd.DataFrame({
    'Algorithm': ['K-Means', 'Hierarchical', 'Gaussian Mixture', 'DBSCAN'],
    'Silhouette Score': [silhouette_kmeans, silhouette_hierarchical, silhouette_gmm, silhouette_dbscan],
    'N Clusters': [optimal_k, optimal_k, optimal_k, n_clusters_dbscan]
})
print(comparison.to_string(index=False))

best_algo_idx = comparison['Silhouette Score'].idxmax()
best_algorithm = comparison.loc[best_algo_idx, 'Algorithm']
print(f"\n🏆 Best Algorithm: {best_algorithm}")

if best_algorithm == 'K-Means':
    final_labels = labels_kmeans
elif best_algorithm == 'Hierarchical':
    final_labels = labels_hierarchical
elif best_algorithm == 'Gaussian Mixture':
    final_labels = labels_gmm
else:
    final_labels = labels_dbscan
    optimal_k = n_clusters_dbscan

# ============================================================================
# STEP 7: 2D VISUALIZATION
# ============================================================================

print("\n" + "="*80)
print("2D VISUALIZATION")
print("="*80)

# t-SNE
print("\nComputing t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000, verbose=1)
features_tsne = tsne.fit_transform(features_pca)

# PCA 2D
print("\nComputing 2D PCA...")
pca_2d = PCA(n_components=2, random_state=42)
features_pca_2d = pca_2d.fit_transform(features_pca)
print(f"2D PCA explained variance: {pca_2d.explained_variance_ratio_.sum():.4f}")

# Plot both
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

scatter1 = axes[0].scatter(features_tsne[:, 0], features_tsne[:, 1], c=final_labels, cmap='tab20', alpha=0.6, s=10, edgecolors='none')
axes[0].set_title(f't-SNE Visualization\n({best_algorithm} Clustering)', fontsize=16, fontweight='bold')
axes[0].set_xlabel('t-SNE Component 1', fontsize=12, fontweight='bold')
axes[0].set_ylabel('t-SNE Component 2', fontsize=12, fontweight='bold')
cbar1 = plt.colorbar(scatter1, ax=axes[0])
cbar1.set_label('Cluster ID', fontsize=12, fontweight='bold')

scatter2 = axes[1].scatter(features_pca_2d[:, 0], features_pca_2d[:, 1], c=final_labels, cmap='tab20', alpha=0.6, s=10, edgecolors='none')
axes[1].set_title(f'PCA 2D Visualization\n({best_algorithm} Clustering)', fontsize=16, fontweight='bold')
axes[1].set_xlabel('First Principal Component', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Second Principal Component', fontsize=12, fontweight='bold')
cbar2 = plt.colorbar(scatter2, ax=axes[1])
cbar2.set_label('Cluster ID', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('05_2d_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# STEP 8: VISUALIZE CLUSTER SAMPLES
# ============================================================================

print("\n" + "="*80)
print("CLUSTER SAMPLE VISUALIZATION")
print("="*80)

samples_per_cluster = 10
fig, axes = plt.subplots(optimal_k, samples_per_cluster, figsize=(20, 2.5*optimal_k))

if optimal_k == 1:
    axes = axes.reshape(1, -1)

for cluster_id in range(optimal_k):
    cluster_mask = final_labels == cluster_id
    cluster_indices = np.where(cluster_mask)[0]
    
    if len(cluster_indices) == 0:
        continue
    
    n_samples_in_cluster = len(cluster_indices)
    
    if n_samples_in_cluster >= samples_per_cluster:
        sample_idx = np.random.choice(cluster_indices, samples_per_cluster, replace=False)
    else:
        sample_idx = cluster_indices
    
    for i in range(samples_per_cluster):
        if i < len(sample_idx):
            axes[cluster_id, i].imshow(images[sample_idx[i]])
            axes[cluster_id, i].axis('off')
            if i == 0:
                axes[cluster_id, i].set_title(f'Cluster {cluster_id}\n({n_samples_in_cluster} images)', 
                                             fontsize=11, fontweight='bold', loc='left')
        else:
            axes[cluster_id, i].axis('off')

plt.suptitle(f'Sample Images from Each Cluster ({best_algorithm})', fontsize=18, fontweight='bold', y=0.998)
plt.tight_layout()
plt.savefig('06_cluster_samples.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# STEP 9: CLUSTER CHARACTERISTICS ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("CLUSTER CHARACTERISTICS ANALYSIS")
print("="*80)

characteristics = []
for cluster_id in range(optimal_k):
    cluster_mask = final_labels == cluster_id
    cluster_images = images[cluster_mask]
    
    if len(cluster_images) == 0:
        continue
    
    avg_brightness = cluster_images.mean()
    std_brightness = cluster_images.std()
    avg_red = cluster_images[:, :, :, 0].mean()
    avg_green = cluster_images[:, :, :, 1].mean()
    avg_blue = cluster_images[:, :, :, 2].mean()
    color_variance = cluster_images.std(axis=(0, 1, 2)).mean()
    
    characteristics.append({
        'Cluster': cluster_id,
        'Size': cluster_mask.sum(),
        'Avg_Brightness': avg_brightness,
        'Std_Brightness': std_brightness,
        'Avg_Red': avg_red,
        'Avg_Green': avg_green,
        'Avg_Blue': avg_blue,
        'Color_Variance': color_variance
    })

cluster_stats = pd.DataFrame(characteristics)
print("\nCluster Statistics:")
print(cluster_stats.to_string(index=False))

# Visualize cluster characteristics
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].bar(cluster_stats['Cluster'], cluster_stats['Size'], color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Cluster ID', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Number of Images', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Cluster Sizes', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='y')

axes[0, 1].bar(cluster_stats['Cluster'], cluster_stats['Avg_Brightness'], color='orange', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Cluster ID', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Average Brightness', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Average Brightness per Cluster', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

x = np.arange(len(cluster_stats))
width = 0.25
axes[1, 0].bar(x - width, cluster_stats['Avg_Red'], width, label='Red', color='red', alpha=0.7, edgecolor='black')
axes[1, 0].bar(x, cluster_stats['Avg_Green'], width, label='Green', color='green', alpha=0.7, edgecolor='black')
axes[1, 0].bar(x + width, cluster_stats['Avg_Blue'], width, label='Blue', color='blue', alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Cluster ID', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Average Color Value', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Average RGB Values per Cluster', fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(cluster_stats['Cluster'])
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

axes[1, 1].bar(cluster_stats['Cluster'], cluster_stats['Color_Variance'], color='purple', edgecolor='black', alpha=0.7)
axes[1, 1].set_xlabel('Cluster ID', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Color Variance', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Color Variance per Cluster', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('07_cluster_characteristics.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# STEP 10: CLUSTER INTERPRETATION
# ============================================================================

print("\n" + "="*80)
print("CLUSTER INTERPRETATION")
print("="*80)

for _, row in cluster_stats.iterrows():
    cluster_id = int(row['Cluster'])
    brightness = row['Avg_Brightness']
    red = row['Avg_Red']
    green = row['Avg_Green']
    blue = row['Avg_Blue']
    
    rgb = [red, green, blue]
    colors = ['Red', 'Green', 'Blue']
    dominant_color = colors[np.argmax(rgb)]
    
    if brightness < 85:
        brightness_cat = "Dark"
    elif brightness < 170:
        brightness_cat = "Medium"
    else:
        brightness_cat = "Bright"
    
    interpretation = f"{brightness_cat} images with {dominant_color}-ish tones"
    
    print(f"\nCluster {cluster_id} ({int(row['Size'])} images):")
    print(f"  - {interpretation}")
    print(f"  - Brightness: {brightness:.1f}/255")
    print(f"  - RGB: R={red:.1f}, G={green:.1f}, B={blue:.1f}")

# ============================================================================
# STEP 11: FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("FINAL PROJECT SUMMARY")
print("="*80)

summary = f"""
PROJECT: Discovering Image Clusters in STL-10 Dataset
METHOD: Unsupervised Machine Learning

DATASET INFORMATION:
  • Total images analyzed: {len(images):,}
  • Image dimensions: {images.shape[1]}×{images.shape[2]}×{images.shape[3]}

FEATURE EXTRACTION:
  • Color Histograms (RGB, 32 bins)
  • HOG (Histogram of Oriented Gradients)
  • Statistical Features
  • Texture Features (LBP)
  • Total features: {all_features.shape[1]:,} dimensions

DIMENSIONALITY REDUCTION:
  • Method: PCA
  • Reduced to: {features_pca.shape[1]} components
  • Variance explained: {pca.explained_variance_ratio_.sum():.2%}

CLUSTERING:
  • Best Algorithm: {best_algorithm}
  • Optimal clusters: {optimal_k}
  • Silhouette Score: {comparison.loc[best_algo_idx, 'Silhouette Score']:.4f}

VISUALIZATION:
  • 2D projections: t-SNE and PCA
  • Cluster sample grids
  • Statistical analysis charts

✅ PROJECT COMPLETE!
"""

print(summary)

with open('project_summary.txt', 'w') as f:
    f.write(summary)
    f.write("\n\nCLUSTER STATISTICS:\n")
    f.write(cluster_stats.to_string(index=False))

print("\n✅ Summary saved to 'project_summary.txt'")
print("✅ All visualizations saved")
print("\n" + "="*80)